In [16]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from ipywidgets import Button, FloatSlider, HBox
from IPython.display import display
from vtkmodules.util import numpy_support
import vtk
import os

In [17]:
DATA_DIR = os.path.join("mixture.vti")

In [18]:
# Function to load VTI file and return data
def load_vti_data(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    image_data = reader.GetOutput()
    point_data = image_data.GetPointData()
    array_data = point_data.GetArray(0)
    numpy_data = numpy_support.vtk_to_numpy(array_data)
    dims = image_data.GetDimensions()
    return numpy_data.reshape(dims, order='F'), dims

In [19]:
# Load your VTI file
volume_data, dims = load_vti_data(DATA_DIR)

# Generate grid
x = np.arange(0, dims[0])
y = np.arange(0, dims[1])
z = np.arange(0, dims[2])
x, y, z = np.meshgrid(x, y, z, indexing='ij')

print("ISOMIN value : {}".format(volume_data.min()))
print("ISOMAX value : {}".format(volume_data.max()))

ISOMIN value : -0.9935540556907654
ISOMAX value : 0.43280163407325745


In [20]:
# Initial parameters
initial_isomin = volume_data.min()
initial_isomax = volume_data.max()
initial_hist_range = np.ones_like(volume_data.flatten(), dtype=bool)

def get_color(isovalue, min_iso, max_iso):
    norm_value = (isovalue - min_iso) / (max_iso - min_iso)
    return [norm_value, 0.5, 1 - norm_value] 

# Function to update plots
def update_plot(isovalue):

    color = get_color(isovalue, np.min(volume_data), np.max(volume_data))

    with isosurface.batch_update():
        isosurface.data[0].isomin = isovalue - 0.25
        isosurface.data[0].isomax = isovalue + 0.25
        # isosurface.data[0].surfacecolor = color 

    with histogram.batch_update():
        hist_data = volume_data.flatten()
        hist_range = (hist_data >= isovalue - 0.25) & (hist_data <= isovalue + 0.25)
        histogram.data[0].x = hist_data[hist_range]

# Create Isosurface and Histogram plots
isosurface = go.FigureWidget(data = [go.Isosurface(
                    x = x.flatten(), y = y.flatten(), z = z.flatten(),
                    value = volume_data.flatten(),
                    colorscale = "Viridis",
                    isomin = 0.0, 
                    isomax = 0.1,
                    surface_count = 1)])

# Setting plot properties
isosurface.update_layout(
    title = "Isosurface Plot",
    width = 500,
    height = 500,
    scene = dict(
        xaxis_title = 'x',
        yaxis_title = 'y',
        zaxis_title = 'z'
    )
)

histogram = go.FigureWidget(data = [go.Histogram(x = volume_data.flatten(), nbinsx = 50)])

# Setting histogram properties
histogram.update_layout(
    title = "Histogram",
    width = 500,
    height = 500,
    xaxis_title = 'Scalar vortex values',
    yaxis_title = 'Frequency'
)

# Slider Widget
slider = FloatSlider(
    value = 0.0, 
    min = np.min(volume_data), 
    max = np.max(volume_data),
    step = 0.01, 
    description = 'Isoval:',
    continuous_update = False
)

slider.observe(lambda change: update_plot(change['new']), 'value')

# Reset Button
reset_button = Button(description="Reset")

def reset_button_clicked(b):
    slider.value = 0.0
    with isosurface.batch_update():
        isosurface.data[0].isomin = 0.0
        isosurface.data[0].isomax = 0.1

    with histogram.batch_update():
        histogram.data[0].x = volume_data.flatten()[initial_hist_range]
    

reset_button.on_click(reset_button_clicked)

In [21]:
# Display everything
top_row = HBox([slider, reset_button])
bottom_row = HBox([isosurface, histogram])
display(top_row, bottom_row)

    'data': [{'colorscale': [[0.0, '#440154'], [0.1111111111111111, '#482878'],
…